In [1]:
import pandas as pd
import re
import string
import numpy as np
import utils 
import nltk


C:\Users\peaks\AppData\Local\Temp\ipykernel_21324\831790845.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [22]:
from keras import models, layers, metrics, optimizers,losses,Sequential,Input,regularizers

In [75]:
from sklearn.model_selection import train_test_split

In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\peaks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
index = df_train.shape[0]

In [5]:
df = pd.concat([df_train,df_test])
text = df['text'].tolist()
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0


In [6]:
def processed(text):
    a = []
    for t in text:
        a.append(' '.join(utils.process_tweet(t)))
    return a


In [7]:
df_processed = processed(text)
df['Processed'] = df_processed

In [8]:
df_processed

['deed reason earthquak may allah forgiv us',
 'forest fire near la rong sask canada',
 'resid ask shelter place notifi offic evacu shelter place order expect',
 '13,000 peopl receiv wildfir evacu order california',
 'got sent photo rubi alaska smoke wildfir pour school',
 'rockyfir updat california hwi 20 close direct due lake counti fire cafir wildfir',
 'flood disast heavi rain caus flash flood street manit colorado spring area',
 "i'm top hill see fire wood ...",
 "there' emerg evacu happen build across street",
 "i'm afraid tornado come area ...",
 'three peopl die heat wave far',
 'haha south tampa get flood hah wait second live south tampa gonna gonna fvck flood',
 "rain flood florida tampabay tampa 18 19 day i'v lost count",
 'flood bago myanmar arriv bago',
 'damag school bu 80 multi car crash break',
 "what' man",
 'love fruit',
 'summer love',
 'car fast',
 'goooaaal',
 'ridicul ...',
 'london cool ;)',
 'love ski',
 'wonder day',
 'loool',
 "way ... can't eat shit",
 'nyc l

In [9]:
df.head()

,id,keyword,location,text,target,Processed
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0,deed reason earthquak may allah forgiv us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0,forest fire near la rong sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0,resid ask shelter place notifi offic evacu she...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0,"13,000 peopl receiv wildfir evacu order califo..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0,got sent photo rubi alaska smoke wildfir pour ...


In [10]:
y = df['target']
X = df['Processed'].tolist()

In [11]:
index = df_train.shape[0]
X_tng = df.loc[df['id'] < index]
x_tst = df.loc[df['id'] >= index]
X_tng = X_tng.drop(columns='id')
x_tst = x_tst.drop(columns='id')

In [12]:
X_t = X_tng['text'].tolist()
X_t = np.array(X_t)

In [13]:
y = y.dropna().tolist()
y
y = np.array(y)
raw_train_das = zip(X_t,y)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X_t, y, test_size=0.2, random_state=69)

In [61]:
VOCAB_SIZE = 10000
encoder = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,standardize='lower_and_strip_punctuation')
encoder.adapt(X_t)

In [15]:
vocab = np.array(encoder.get_vocabulary())
len(vocab)

1000

In [70]:

model = Sequential([
    encoder,
    layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    layers.Bidirectional(layers.LSTM(64,  return_sequences=True)),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1,activation='sigmoid',kernel_regularizer=regularizers.L2(0.0005))
])


In [62]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True, True, True, True, True]


In [43]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

1/1 [==============================] - 2s 2s/step
[0.5012194]


In [37]:
padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

1/1 [==============================] - 0s 102ms/step
[0.4961412]


In [71]:
model.compile(loss="binary_crossentropy",
              optimizer=optimizers.Adam(learning_rate=9e-3),
              metrics=['accuracy'])

In [72]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (Text  (None, None)              0         
 Vectorization)                                                  
                                                                 
 embedding_10 (Embedding)    (None, None, 64)          640000    
                                                                 
 bidirectional_10 (Bidirect  (None, None, 128)         66048     
 ional)                                                          
                                                                 
 bidirectional_11 (Bidirect  (None, 64)                41216     
 ional)                                                          
                                                                 
 dense_20 (Dense)            (None, 64)                4160      
                                                      

In [78]:
history = model.fit(X_t,y, epochs=5,validation_data=(X_test,y_test),validation_steps=30)

Epoch 1/5
238/238 [==============================] - 17s 69ms/step - loss: 0.3750 - accuracy: 0.8660 - val_loss: 0.2055 - val_accuracy: 0.9258
Epoch 2/5
238/238 [==============================] - 16s 69ms/step - loss: 0.2335 - accuracy: 0.9124 - val_loss: 0.1245 - val_accuracy: 0.9481
Epoch 3/5
238/238 [==============================] - 17s 71ms/step - loss: 0.1617 - accuracy: 0.9339 - val_loss: 0.1015 - val_accuracy: 0.9540
Epoch 4/5
238/238 [==============================] - 17s 71ms/step - loss: 0.1304 - accuracy: 0.9440 - val_loss: 0.0860 - val_accuracy: 0.9606
Epoch 5/5
238/238 [==============================] - 17s 71ms/step - loss: 0.1141 - accuracy: 0.9513 - val_loss: 0.0834 - val_accuracy: 0.9645


Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 68s 339ms/step - accuracy: 0.5792 - loss: 0.6891 - val_accuracy: 0.5345 - val_loss: 0.6913
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 64s 336ms/step - accuracy: 0.5706 - loss: 0.6619 - val_accuracy: 0.5391 - val_loss: 0.7715
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 62s 325ms/step - accuracy: 0.7348 - loss: 0.5187 - val_accuracy: 0.5469 - val_loss: 0.8458
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 62s 323ms/step - accuracy: 0.7824 - loss: 0.4492 - val_accuracy: 0.5430 - val_loss: 0.9157
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 63s 332ms/step - accuracy: 0.8118 - loss: 0.4135 - val_accuracy: 0.5456 - val_loss: 1.0228

In [53]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail(1)

,loss,accuracy,val_loss,val_accuracy,epoch
19,0.095384,0.958621,3.962561,0.510177,19


In [54]:
x_t = np.array(x_tst['Processed'].tolist())

In [55]:
model.evaluate(x_t)

102/102 [==============================] - 2s 901us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

In [57]:
res = model.predict(x_t)

102/102 [==============================] - 0s 3ms/step


In [67]:
result = []
for i in res:
    if i >= 0.5:
        result.append(1)
    else:
        result.append(0)
len(result)

3263

In [65]:
df_res = df_test.drop(columns=['keyword','location','text'])
df_res['target'] = result

In [68]:
df_res.to_csv('submission.csv', index=False)
df_res.to_csv(r'C:\Users\peaks\Desktop\Submission\submission.csv', index=False)